In [58]:
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import numpy as np
from urllib.parse import urljoin

url_home = "https://swgoh.gg/"

player_id = "559143933"
toon = "jedi-master-luke-skywalker"

def fun_toon_stats(url):
    # Initialisiere das Dictionary für die Ergebnisse
    ergebnisse_dict = {}
    
    # HTML-Seite abrufen
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.content
    else:
        print("Fehler beim Abrufen der Website.")
        exit()
    
    # Relevanten HTML-Code finden (gemeinsames Skript)
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Finde das Skript-Tag, das die Datenpunkte für alle Diagramme enthält
    data_script = soup.find("script", string=re.compile(r"new Highcharts.Chart\({\s*chart: {\s*renderTo: 'character-stats-"))
    
    # Extrahiere den Inhalt des Skripts
    data_script_content = data_script.string
    
    # Finde alle Highcharts.Chart-Aufrufe im data_script
    chart_aufraeufe = re.findall(r"new Highcharts.Chart\({(.*?)}\)", data_script_content, re.DOTALL)
    
    # Annahme: Du hast bereits chart_aufraeufe aus dem data_script extrahiert
    for aufrauf in chart_aufraeufe:
    
         # Extrahiere den Titel aus dem Aufrauf
        title_start = aufrauf.find("title: {") + len("title: {")
        title_end = aufrauf.find("}", title_start)
        title_string = aufrauf[title_start:title_end]
    
        # Extrahiere den Text des Titels
        title_match = re.search(r"text: '(.*?)'", title_string)
        if title_match:
            diagramm_titel = title_match.group(1).split(" (Nearest ")[0]  # Entferne den Zusatz
        else:
            diagramm_titel = "Custom Diagram Title"  # Fallback-Titel, falls kein Text gefunden wird
    
            
        # Extrahiere die Datenpunkte aus dem String
        data_start = aufrauf.find("data: [") + len("data: [")
        data_end = aufrauf.find("]", data_start)
        data_string = aufrauf[data_start:data_end]
    
        # Teile den data_string in einzelne Datenpunkte auf
        datenpunkte = data_string.split("},\n")
    
        # Extrahiere x- und y-Wertepaare
        x_werte = []
        y_werte = []
        for punkt in datenpunkte:
            x_start = punkt.find("x:") + len("x:")
            x_end = punkt.find(",", x_start)
            x = punkt[x_start:x_end].strip()
    
            y_start = punkt.find("y:") + len("y:")
            y_end = punkt.find(",", y_start)
            y = punkt[y_start:y_end].strip()
    
            if x:  # Prüfe, ob der x-Wert nicht leer ist
                x_werte.append(float(x))
            if y:  # Prüfe, ob der y-Wert nicht leer ist
                y_werte.append(float(y))  # Ändere y zu float
        
        # Berechne den gewichteten Mittelwert und die gewichtete Standardabweichung
        x_array = np.array(x_werte)
        y_array = np.array(y_werte)
        gewichteter_mittelwert = np.average(x_array, weights=y_array)
        gewichtete_std = np.sqrt(np.average((x_array - gewichteter_mittelwert) ** 2, weights=y_array))
        # Berechne das Verhältnis SD/Mean
    
        # Speichere die Ergebnisse im Dictionary
        ergebnisse_dict[diagramm_titel] = [gewichteter_mittelwert, gewichtete_std]
    
    # Gib das Ergebnis aus
    return ergebnisse_dict

def fun_toon_base(url):
    # HTML-Seite abrufen
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.content
    else:
        print("Fehler beim Abrufen der Website.")
        exit()
    
    # Relevanten HTML-Code finden (gemeinsames Skript)
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Angenommen, du hast bereits die 'soup' mit den Daten
    # Finde das 'window.unit_data_lookup'-Objekt
    unit_data_lookup_script = soup.find("script", string=re.compile(r"window.unit_data_lookup"))
    if unit_data_lookup_script:
        # Extrahiere den Textinhalt des Skripts
        unit_data_lookup_text = unit_data_lookup_script.text
    
        # Suche nach dem GEAR_13-Teil im 'window.unit_data_lookup'
        gear_13_start = unit_data_lookup_text.find("GEAR_13:")
        if gear_13_start != -1:
            gear_13_text = unit_data_lookup_text[gear_13_start:]
    
            # Extrahiere die relevanten Werte
            relevant_values = ["Health", "Speed", "Protection", "Tenacity", "Potency", "Armor", "Resistance"]
            gear_13_data = {}
            for value in relevant_values:
                value_start = gear_13_text.find(f'"label": "{value}"')
                if value_start != -1:
                    # Suche nach dem Wert für "value"
                    value_value_start = gear_13_text.find('"value": ', value_start)
                    value_value_end = gear_13_text.find(",", value_value_start)
                    value_str = gear_13_text[value_value_start + len('"value": '):value_value_end]
                    try:
                        gear_13_data[value] = float(value_str)  # Wir verwenden float, da einige Werte Dezimalstellen haben
                    except ValueError:
                        print(f"Ungültiger Wert für {value}: {value_str}")
                else:
                    print(f"{value} nicht gefunden.")
            for section in ["Physical Offense", "Special Offense"]:
                section_start = gear_13_text.find(f'"label": "{section}"')
                if section_start != -1:
                    # Suche nach dem Wert für "Damage"
                    damage_start = gear_13_text.find('"label": "Damage"', section_start)
                    if damage_start != -1:
                        damage_value_start = gear_13_text.find('"value": ', damage_start)
                        damage_value_end = gear_13_text.find(",", damage_value_start)
                        damage_str = gear_13_text[damage_value_start + len('"value": '):damage_value_end]
                        try:
                             gear_13_data["Attack Damage" if section == "Physical Offense" else "Special Damage"] = float(damage_str)
                        except ValueError:
                            print(f"Ungültiger Wert für {section} Damage: {damage_str}")
                    else:
                        print(f"Damage für {section} nicht gefunden.")
                else:
                    print(f"{section} nicht gefunden.")
    
            
        else:
            print("GEAR_13 nicht gefunden.")
    else:
        print("window.unit_data_lookup nicht gefunden.")
    
    return gear_13_data

def fun_toon_player(url):
    # HTML-Seite abrufen
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.content
    else:
        print("Fehler beim Abrufen der Website.")
        exit()
    
    # Relevanten HTML-Code finden
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Extrahiere die Werte
    def extract_stat(label):
        stat_element = soup.find("span", class_="unit-stat-group-stat-label", string=label)
        if stat_element:
            value_element = stat_element.find_next("span", class_="unit-stat-group-stat-value")
            if value_element:
                value_text = value_element.text.strip()
                # Extrahiere den Wert (ohne Ausrüstung)
                value = value_text.split("(")[0].replace(",", "").strip()  # Entferne Tausendertrennzeichen
                if "%" in value:
                    # Prozentwert in Fließkommazahl umwandeln
                    value = float(value.replace("%", ""))
                    if label in ["Tenacity", "Potency"]:
                        value /= 100  # Teile durch 100
                else:
                    value = float(value)
                return value
        return None
    
    # Daten in einer Dictionary-Struktur speichern
    data = {
        "Health": extract_stat("Health"),
        "Speed": extract_stat("Speed"),
        "Protection": extract_stat("Protection"),
        "Tenacity": extract_stat("Tenacity"),
        "Potency": extract_stat("Potency"),
        "Armor": extract_stat("Armor"),
        "Resistance": extract_stat("Resistance"),
        "Attack Damage": extract_stat("Physical Damage"),
        "Special Damage": extract_stat("Special Damage")
    }
    
    return data

def get_toon_score(player_id, toon):
    
    # Kombiniere die Basis-URL mit dem relativen Pfad zum Charakter
    url_toon_player = urljoin(url_home, f"p/{player_id}/characters/{toon}")
    url_toon_base = urljoin(url_home, f"characters/{toon}") 
    url_toon_stats = urljoin(url_home, f"characters/{toon}/data/stats?filter_type=guild_100_gp") 
            
    toon_stats = fun_toon_stats(url_toon_stats)    
    toon_base = fun_toon_base(url_toon_base)   
    toon_player = fun_toon_player(url_toon_player)
    
    # Führen Sie die Einträge aus toon_base und toon_player in toon_stats zusammen
    for key in toon_stats:
        toon_stats[key].append(toon_base[key])
        toon_stats[key].append(toon_player[key])
    
    gewichte = []
    wertung = []
    
    for key in toon_stats:
        values = toon_stats[key]
        gewichte.append( (values[0] - values[2])/values[2] )
        wertung.append( (values[3] - values[0])/values[1] )
    
    gewichte = np.array(gewichte)
    gewichte = gewichte/np.sum(gewichte)
    wertung = np.array(wertung)
    
    scores = gewichte*wertung
    
    score = np.sum(scores)

    return score

player_id = "559143933"
toon = "jedi-master-luke-skywalker"

#player_id = "382771759"
#player_id = "883415444"

score = get_toon_score(player_id, toon)

print(score)


-0.7525421968572644
